In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
#훈련 데이터셋 로드
training_data=datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [3]:
#테스트 데이터셋 로드
test_data=datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size=512

train_dataloader= DataLoader(training_data, batch_size=batch_size)
test_dataloader= DataLoader(test_data, batch_size=batch_size)


In [5]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y:{y.shape}, Type is '{y.dtype}'")
    break

Shape of X [N, C, H, W]: torch.Size([512, 1, 28, 28])
Shape of y:torch.Size([512]), Type is 'torch.int64'


In [6]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 모델 정의

In [8]:
class CnnBasedFashion(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.fc_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.conv_stack(x)
        x = self.fc_stack(x)
        return x


model = CnnBasedFashion().to(device=device)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)
    
print(model)

Using 2 GPUs
DataParallel(
  (module): CnnBasedFashion(
    (conv_stack): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc_stack): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=3136, out_features=128, bias=True)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=10, bias=True)
    )
  )
)


## 모델학습 정의

In [9]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #오차 계산
        pred=model(X)
        loss=loss_func(pred, y)
        
        #Back-propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"los: {loss:>7f} [{current:>5d}/{size:5>5d}]")

In [11]:
def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches =len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred=model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg Loss: {test_loss:>8f} \n")

## 모델 학습시키기

In [12]:
epoch = 10

for t in range(epoch):
    print(f"Epoch: {t+1}\n---------------------------------------")
    train(train_dataloader, model, loss_func=loss_func, optimizer=optimizer)
    test(test_dataloader, model, loss_func)
print("Done!")

Epoch: 1
---------------------------------------
los: 2.299943 [  512/60000]
los: 2.292825 [51712/60000]
Test Error: 
 Accuracy: 10.0%, Avg Loss: 2.293397 

Epoch: 2
---------------------------------------
los: 2.292573 [  512/60000]
los: 2.287254 [51712/60000]
Test Error: 
 Accuracy: 10.0%, Avg Loss: 2.288083 

Epoch: 3
---------------------------------------
los: 2.287365 [  512/60000]
los: 2.281882 [51712/60000]
Test Error: 
 Accuracy: 10.4%, Avg Loss: 2.282730 

Epoch: 4
---------------------------------------
los: 2.282161 [  512/60000]
los: 2.276350 [51712/60000]
Test Error: 
 Accuracy: 12.3%, Avg Loss: 2.277068 

Epoch: 5
---------------------------------------
los: 2.276672 [  512/60000]
los: 2.270001 [51712/60000]
Test Error: 
 Accuracy: 14.7%, Avg Loss: 2.270548 

Epoch: 6
---------------------------------------
los: 2.270337 [  512/60000]
los: 2.262621 [51712/60000]
Test Error: 
 Accuracy: 17.3%, Avg Loss: 2.262940 

Epoch: 7
---------------------------------------
los: 2.26

batch=32, epoch=10일 경우 소요시간 1분 55초 / 407mb gram 소요

batch=64, epoch=10 일 경우 소요시간 1분 37초 / 409mb gram 소요

batch=128, epoch=10 일 경우 소요시간 1분 25초 / 409mb gram 소요

## 모델 저장하기

In [13]:
torch.save(model.state_dict(), "checkpoints/cnn_model.pth")
print("Model saved...")

Model saved...


## 모델 불러오기

In [17]:
model =CnnBasedFashion()
model =nn.DataParallel(model)
model.to(device=device)
model.load_state_dict(torch.load("checkpoints/cnn_model.pth"))

<All keys matched successfully>

## 모델 추론하기

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
index = 90
x, y = test_data[index][0], test_data[index][1]
with torch.no_grad():
    x = x.unsqueeze(0).to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Sandal", Actual: "Sandal"


In [ ]:
"""def clear_gpu_memory():
    import gc
    import torch

    globals_to_clear = ['model', 'x', 'y', 'pred']  # 전역 네임스페이스에 등록된 이름도 삭제
    for name in globals_to_clear:
        if name in globals():
            globals()[name] = None

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("🧹 GPU memory cleared!")

clear_gpu_memory()"""